In [1]:
import torch
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('training_samples.csv')
df['label'] = df['label'].str.lower()
test_df = df[1600:]
X_test = test_df['text']
y_test = test_df['label']

### 1. Vader Sentiment

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    sentiment = analyser.polarity_scores(text)
    sentiment = sentiment['compound']
    if sentiment <= -0.05:
        return 'negative'
    elif sentiment > -0.05 and sentiment < 0.05:
        return 'neutral'
    elif sentiment >= 0.05:
        return 'positive'

In [5]:
labels = ['negative', 'neutral', 'positive', 'unrelated']

test_df['vader_sentiment'] = test_df['text'].apply(vader_sentiment)
y_pred = test_df['vader_sentiment']
print("test accuracy: %.2f%%" % (accuracy_score(y_pred, y_test)*100))
print(classification_report(y_test, y_pred, target_names=labels))

test accuracy: 55.75%
              precision    recall  f1-score   support

    negative       0.86      0.50      0.64       165
     neutral       0.07      0.42      0.11        19
    positive       0.73      0.69      0.71       190
   unrelated       0.00      0.00      0.00        26

    accuracy                           0.56       400
   macro avg       0.41      0.40      0.36       400
weighted avg       0.70      0.56      0.60       400



### 2. RoBERTa-based sentiment

In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

#Load the pre-trained model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def roberta_polarity_scores(text):
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive': scores[2]
    }
    return scores_dict

def roberta_sentiment(text):
    try:
        scores_dict = roberta_polarity_scores(text)
        sentiment = max(scores_dict, key=scores_dict.get)
        return sentiment
    except:
        return ""

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
labels = ['negative', 'neutral', 'positive', 'unrelated']

test_df['roberta_sentiment'] = test_df['text'].apply(roberta_sentiment)
y_pred = test_df['roberta_sentiment']
print("test accuracy: %.2f%%" % (accuracy_score(y_pred, y_test)*100))
print(classification_report(y_test, y_pred, target_names=labels))


test accuracy: 62.00%
              precision    recall  f1-score   support

    negative       0.90      0.65      0.75       165
     neutral       0.12      0.84      0.21        19
    positive       0.87      0.66      0.75       190
   unrelated       0.00      0.00      0.00        26

    accuracy                           0.62       400
   macro avg       0.47      0.54      0.43       400
weighted avg       0.79      0.62      0.68       400

